LSTM on dcdc converter

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt
# plot param
%matplotlib inline
plt.rcParams['figure.figsize'] = (22.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

First, manipulate the input and output data

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [2]:
# parse the controller file
f = open('../COTONN/dcdc_small/controller.scs', "r")

In [3]:
# parse the controller file
# f = open('controller.scs', "r")

Skip the header part and go straight to the state-action pair

In [4]:
lines = []
for line in f:
    if '#MATRIX:DATA\n' in line:                
        for line in f: # now you are at the lines you want
            # skip the #BEGIN 
            # read the state-actions
            lines = f.readlines()

In [5]:
del lines[-1]

In [6]:
# take the state as the train dataset
ltrain_dataset = []
for x in lines:
    ltrain_dataset.append(x.split(' ')[0])
# del ltrain_dataset[-1] # delete the string #END at the end of the file 

In [7]:
# convert to the numpy array with float32 data type
train_dataset = np.asarray(ltrain_dataset)
train_dataset = train_dataset.astype(np.float32)
train_dataset

array([  188.,   189.,   190., ..., 10080., 10081., 10082.], dtype=float32)

In [8]:
# take the action(s) [column 1:-1] / the rest of the integer except the state
ltrain_label = []
for x in lines:
    # ltrain_label.append(x.split(' ')[1:-1])
    ltrain_label.append(x.split(' ')[1].strip('\n'))
# del ltrain_label[-1] # delete the string #END at the end of the file  

In [67]:
ltrain_label

['1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0'

In [9]:
# convert to numpy array, note that the result is still not in one hot encoding format
train_label = np.asarray(ltrain_label)
train_label

array(['1', '1', '1', ..., '0', '0', '0'], dtype='<U1')

Iterate from here

In [10]:
train_dataset

array([  188.,   189.,   190., ..., 10080., 10081., 10082.], dtype=float32)

In [11]:
# define number of samples
num_samples = train_dataset.shape[0]
num_samples

8677

In [12]:
# create now array to be filled by the encoded label
train_label_hot = np.zeros([num_samples,2], dtype=np.float32)
train_label_hot.shape

(8677, 2)

In [13]:
# encode label to one hot encoding format
for i in range(num_samples):
    if train_label[i] == ['0']:
        train_label_hot[i] = [1, 0]
    elif train_label[i] == ['1']:
        train_label_hot[i] = [0, 1]
    elif train_label[i] == ['0','1']:
        train_label_hot[i] = [1, 1]

In [14]:
train_label_hot[75]

array([0., 0.], dtype=float32)

Execute this to get the full batch

In [15]:
# transform the dimension of the dataset
train_dataset_reform = train_dataset[:, None]
train_dataset_reform.shape

(8677, 1)

# NN

In [16]:
def accuracy_softmax(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [17]:
def accuracy_multilabel(predictions, labels):
  return (100.0 * np.mean(np.asarray(predictions) == np.asarray(labels)))

In [18]:
train_dataset_reform[0]

array([188.], dtype=float32)

In [19]:
train_dataset_reform[-1]

array([10082.], dtype=float32)

In [20]:
num_samples

8677

In [21]:
#TRANSFORM

In [22]:
train_dataset_reform_dim = train_dataset[:, None]
train_dataset_reform_dim

array([[  188.],
       [  189.],
       [  190.],
       ...,
       [10080.],
       [10081.],
       [10082.]], dtype=float32)

In [23]:
upper_limit = train_dataset_reform_dim.shape[0]
upper_limit

8677

In [24]:
# slice the samples
train_sliced = train_dataset_reform_dim[:upper_limit]
train_sliced

array([[  188.],
       [  189.],
       [  190.],
       ...,
       [10080.],
       [10081.],
       [10082.]], dtype=float32)

In [25]:
# change it to init
train_sliced_int = train_sliced.astype(np.uint32)
train_sliced_int

array([[  188],
       [  189],
       [  190],
       ...,
       [10080],
       [10081],
       [10082]], dtype=uint32)

In [26]:
tsi8_unordered = train_sliced_int.view(np.uint8)
tsi8 = np.flip(tsi8_unordered,1)
tsi8

array([[  0,   0,   0, 188],
       [  0,   0,   0, 189],
       [  0,   0,   0, 190],
       ...,
       [  0,   0,  39,  96],
       [  0,   0,  39,  97],
       [  0,   0,  39,  98]], dtype=uint8)

In [27]:
# converte to bit representation
# toy_vehicle_input = np.unpackbits(tu_u8, axis=1)
# toy_vehicle_input
toy_vehicle_input_unreduced = np.unpackbits(tsi8).reshape(-1,32)
toy_vehicle_input_unreduced

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]], dtype=uint8)

In [28]:
toy_vehicle_input = toy_vehicle_input_unreduced[:, np.arange(16,32)]

In [29]:
toy_vehicle_input.shape

(8677, 16)

In [30]:
#LABEL

In [31]:
train_label_reform = train_label[:, None].astype(np.uint8)
train_label_reform.shape

(8677, 1)

In [32]:
train_dataset_reform = toy_vehicle_input

In [33]:
train_dataset_reform.shape

(8677, 16)

In [34]:
num_samples

8677

In [35]:
num_classes = train_label_reform.shape[1]
# num_classes = train_label_hot.shape[1]

In [36]:
num_classes

1

In [37]:
# state_size = train_dataset_reform.shape[1]
state_size = 20

In [38]:
total_series_length = num_samples
truncated_backprop_length = train_dataset_reform.shape[1]

In [39]:
batch_size = total_series_length
num_batches = total_series_length//batch_size

In [40]:
num_batches

1

In [41]:
x = train_dataset_reform.reshape((batch_size,-1))
y = train_label_reform.reshape((batch_size, num_classes))
# y = train_label_hot.reshape((batch_size, num_classes))

In [42]:
y.shape

(8677, 1)

In [43]:
batch_size

8677

In [44]:
truncated_backprop_length

16

In [45]:
num_batches

1

In [46]:
x.shape

(8677, 16)

In [47]:
y.shape

(8677, 1)

In [48]:
tf.reset_default_graph()

In [49]:
num_classes

1

In [50]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.float32, [batch_size, num_classes])

# init_state = tf.placeholder(tf.float32, [batch_size, state_size])
cell_state = tf.placeholder(tf.float32, [batch_size, state_size])
hidden_state = tf.placeholder(tf.float32, [batch_size, state_size])
init_state = tf.nn.rnn_cell.LSTMStateTuple(cell_state, hidden_state)

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

In [51]:
# Unpack columns
inputs_series = tf.split(batchX_placeholder, truncated_backprop_length, 1)
labels_series = batchY_placeholder

In [52]:
inputs_series

[<tf.Tensor 'split:0' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:1' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:2' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:3' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:4' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:5' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:6' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:7' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:8' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:9' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:10' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:11' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:12' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:13' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:14' shape=(8677, 1) dtype=float32>,
 <tf.Tensor 'split:15' shape=(8677, 1) dtype=float32>]

In [53]:
init_state

LSTMStateTuple(c=<tf.Tensor 'Placeholder_2:0' shape=(8677, 20) dtype=float32>, h=<tf.Tensor 'Placeholder_3:0' shape=(8677, 20) dtype=float32>)

In [54]:
from tensorflow.contrib import rnn 

In [55]:
# Forward passes
# cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
# states_series, current_state = tf.nn.rnn(cell, inputs_series, init_state)
# cell = tf.nn.rnn_cell.BasicLSTMCell(state_size, state_is_tuple=True)
cell = rnn.BasicLSTMCell(state_size)

In [56]:
# states_series, current_state = tf.nn.rnn(cell, inputs_series, init_state)
states_series, current_state = rnn.static_rnn(cell, inputs_series, init_state)

In [57]:
# logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
logits = tf.matmul(states_series[-1], W2) + b2 #Broadcasted addition

# predictions_series = [tf.nn.softmax(logits) for logits in logits_series]
predictions = tf.round(tf.sigmoid(logits))

losses = tf.nn.sigmoid_cross_entropy_with_logits(logits = logits,labels = labels_series)
total_loss = tf.reduce_mean(losses)

# train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)
train_step = tf.train.AdamOptimizer(0.01).minimize(total_loss)

In [58]:
logits

<tf.Tensor 'add:0' shape=(8677, 1) dtype=float32>

In [59]:
cell.state_size

LSTMStateTuple(c=20, h=20)

In [60]:
init_state

LSTMStateTuple(c=<tf.Tensor 'Placeholder_2:0' shape=(8677, 20) dtype=float32>, h=<tf.Tensor 'Placeholder_3:0' shape=(8677, 20) dtype=float32>)

In [61]:
states_series

[<tf.Tensor 'rnn/basic_lstm_cell/Mul_2:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_5:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_8:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_11:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_14:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_17:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_20:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_23:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_26:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_29:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_32:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_35:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rnn/basic_lstm_cell/Mul_38:0' shape=(8677, 20) dtype=float32>,
 <tf.Tensor 'rn

In [62]:
current_state

LSTMStateTuple(c=<tf.Tensor 'rnn/basic_lstm_cell/Add_31:0' shape=(8677, 20) dtype=float32>, h=<tf.Tensor 'rnn/basic_lstm_cell/Mul_47:0' shape=(8677, 20) dtype=float32>)

In [63]:
x

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 1, 0]], dtype=uint8)

In [64]:
y

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

In [65]:
x[1,0:16]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1], dtype=uint8)

In [66]:
loss_history = []

acc = 0
acc_cum = 0
acc_temp = 0
epoch_idx = 0

with tf.Session() as sess:
    # sess.run(tf.initialize_all_variables())
    sess.run(tf.global_variables_initializer())
    # plt.ion()
    # plt.figure()
    # plt.show()
    loss_list = []
    print("Initialized")
    while acc < 100:
    # for epoch_idx in range(1000):
        epoch_idx += 1
        acc_cum = 0
        _current_cell_state = np.zeros((batch_size, state_size))
        _current_hidden_state = np.zeros((batch_size, state_size))

        # print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            """
            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })
            """

            _total_loss, _train_step, _current_state, _predictions_series, _labels_series = sess.run(
                [total_loss, train_step, current_state, predictions, labels_series],
                feed_dict={
                    batchX_placeholder: batchX,
                    batchY_placeholder: batchY,
                    cell_state: _current_cell_state,
                    hidden_state: _current_hidden_state
                })

            _current_cell_state, _current_hidden_state = _current_state

            loss_list.append(_total_loss)
            # print(_predictions_series)
            # print(_labels_series)

            acc_temp =  accuracy_multilabel(_predictions_series, _labels_series)
            acc_cum += acc_temp
            acc = acc_cum/(batch_idx+1)
            print("Epoch {0:6d} Step: {1:6d} Loss: {2:8.4f} Training Acc: {3:8.2f} {4:8.2f} {5:8.2f}%".format(epoch_idx, batch_idx, _total_loss, acc_temp, acc_cum, acc))

"""
    # accuracy
    print("TESTING")
    # _current_state = np.zeros((batch_size, state_size))
    _current_cell_state = np.zeros((batch_size, state_size))
    _current_hidden_state = np.zeros((batch_size, state_size))
    acc = 0
    total_acc = 0
    for batch_idx in range(num_batches):
        start_idx = batch_idx * truncated_backprop_length
        end_idx = start_idx + truncated_backprop_length
        
        batchX = x[:,start_idx:end_idx]
        batchY = y[:,start_idx:end_idx]
        
        _current_state, _predictions_series, _labels_series = sess.run(
                [current_state, predictions_series, labels_series],
                feed_dict={
                    batchX_placeholder: batchX,
                    batchY_placeholder: batchY,
                    cell_state: _current_cell_state,
                    hidden_state: _current_hidden_state
                })
            
        _current_cell_state, _current_hidden_state = _current_state
            
        acc = accuracy_independent( _predictions_series, _labels_series)
        print("batch: {0:5d} Accuracy: {1:4.2f}".format(batch_idx,acc))
        total_acc += acc
        print(_predictions_series)
        print(_labels_series)
    
    print("Accuracy: {0:4.2f}%".format(total_acc/num_batches))
# plt.ioff()1
# plt.show()
"""

Initialized


InternalError: Blas GEMM launch failed : a.shape=(8677, 21), b.shape=(21, 80), m=8677, n=80, k=21
	 [[Node: rnn/basic_lstm_cell/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](rnn/basic_lstm_cell/concat, rnn/basic_lstm_cell/kernel/read)]]
	 [[Node: rnn/basic_lstm_cell/Add_31/_13 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1561_rnn/basic_lstm_cell/Add_31", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'rnn/basic_lstm_cell/MatMul', defined at:
  File "c:\users\gebruiker\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\gebruiker\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\gebruiker\anac

In [157]:
_predictions_series

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
_labels_series